<a href="https://colab.research.google.com/github/rsr2425/word-count-investigation/blob/main/notebooks/2_Fine_tuning_for_Word_Counts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
import os

import torch
import torch.nn as nn
import bitsandbytes as bnb
import inspect

from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
import inspect
inspect.signature(model.forward)

<Signature (input_ids: torch.LongTensor = None, attention_mask: Optional[torch.Tensor] = None, position_ids: Optional[torch.LongTensor] = None, past_key_values: Union[transformers.cache_utils.Cache, List[torch.FloatTensor], NoneType] = None, inputs_embeds: Optional[torch.FloatTensor] = None, labels: Optional[torch.LongTensor] = None, use_cache: Optional[bool] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, return_dict: Optional[bool] = None, cache_position: Optional[torch.LongTensor] = None, num_logits_to_keep: int = 0, **kwargs: typing_extensions.Unpack[transformers.models.llama.modeling_llama.KwargsForCausalLM]) -> Union[Tuple, transformers.modeling_outputs.CausalLMOutputWithPast]>

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful summary chatbot.  Summarize the text provided by the user.
The word count of your response should be less than or equal to the value specified in the WORD_COUNT/END_WORD_COUNT block.<|eot_id|><|start_header_id|>user<|end_header_id|>
{text}

[WORD_COUNT]
{target_word_count}
[END_WORD_COUNT]<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

RESPONSE_TEMPLATE = """\
{summary}<|eot_id|><|end_of_text|>"""

In [ ]:
def create_prompt(sample, include_response = True):
  """
  Parameters:
    - sample: dict representing row of dataset
    - include_response: bool

  Functionality:
    This function should build the Python str `full_prompt`.

    If `include_response` is true, it should include the translation -
    else it should not contain the translation (useful for prompting) and testing

  Returns:
    - full_prompt: str
  """
  full_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(text=sample["text"], target_word_count=sample["target_word_count"])

  if include_response:
    full_prompt += RESPONSE_TEMPLATE.format(summary=sample["summary"])

  return full_prompt

In [ ]:
def generate_response(prompt, model, tokenizer):
  """
  Parameters:
    - prompt: str representing formatted prompt
    - model: model object
    - tokenizer: tokenizer object

  Functionality:
    This will allow our model to generate a response to a prompt!

  Returns:
    - str response of the model
  """

  # convert str input into tokenized input
  encoded_input = tokenizer(prompt,  return_tensors="pt")

  # send the tokenized inputs to our GPU
  model_inputs = encoded_input.to('cuda')

  # generate response and set desired generation parameters
  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=256,
      do_sample=True,
      pad_token_id=tokenizer.eos_token_id
  )

  # decode output from tokenized output to str output
  decoded_output = tokenizer.batch_decode(generated_ids)

  # return only the generated response (not the prompt) as output
  return decoded_output[0].split("<|end_header_id|>")[-1]

In [ ]:
from datasets import load_dataset

WORD_COUNT_TARGET = 50

# dataset = load_dataset("billsum", split="ca_test")
# dataset.drop_column('title')
dataset = load_dataset("ccdv/cnn_dailymail", '3.0.0', split={
    'train': 'train[:1000]',
    'validation': 'validation[:1000]',
    'test': 'test[:1000]'
})
dataset = dataset.rename_column('article', 'text')
dataset = dataset.rename_column('highlights', 'summary')
dataset = dataset.remove_columns(['id'])

dataset = dataset.map(lambda x: {'target_word_count': WORD_COUNT_TARGET})

In [ ]:
from peft import LoraConfig, get_peft_model

# set our rank (higher value is more memory/better performance)
lora_r = 16

# set our dropout (default value)
lora_dropout = 0.1

# rule of thumb: alpha should be (lora_r * 2)
lora_alpha = 32

# construct our LoraConfig with the above hyperparameters
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM" # TODO maybe need to change this?
)
model = get_peft_model(
    model,
    peft_config
)

In [ ]:
from trl import SFTConfig

max_seq_length = 1024

args = SFTConfig(
  output_dir = "llama3181binstruct_word_count",
  num_train_epochs=5,
  # max_steps = 100, # comment out this line if you want to train in epochs
  per_device_train_batch_size = 1,
  warmup_steps = 30,
  logging_steps=10,
  evaluation_strategy="epoch",
  # eval_strategy="steps",
  eval_steps=25, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=2e-4,
  lr_scheduler_type='cosine',
  dataset_kwargs={
        "add_special_tokens" : False,
        "append_concat_token" : False,
  },
  max_seq_length=max_seq_length,
  packing=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from trl import SFTTrainer

from torch.autograd import Variable

def count_words(text):
    return len(text.split())

class WordCountLossTrainer(SFTTrainer):
    def __init__(self, *args, **kwargs):
        super(WordCountLossTrainer, self).__init__(*args, **kwargs)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Forward pass
        outputs = model(**inputs)

        # Extract input IDs and logits
        input_ids = inputs["input_ids"]
        logits = outputs.logits

        # Decode input prompts
        tokenizer = self.data_collator.tokenizer if hasattr(self.data_collator, 'tokenizer') else self.tokenizer
        decoded_inputs = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

        # Decode generated responses
        predicted_tokens = torch.argmax(logits, dim=-1)
        decoded_responses = tokenizer.batch_decode(predicted_tokens, skip_special_tokens=True)

        # Extract word count targets
        word_count_targets = []
        for decoded_input in decoded_inputs:
            if "[WORD_COUNT]" in decoded_input and "[END_WORD_COUNT]" in decoded_input:
                start = decoded_input.find("[WORD_COUNT]") + len("[WORD_COUNT]")
                end = decoded_input.find("[END_WORD_COUNT]")
                try:
                    word_count = int(decoded_input[start:end].strip())
                except Exception as e:
                    problematic_example = tokenizer.decode(input_ids[0], skip_special_tokens=True)
                    print(f"Error occurred for input: {problematic_example}")
                    print(f"Error details: {str(e)}")
                word_count_targets.append(WORD_COUNT_TARGET)
            else:
                word_count_targets.append(0)  # Default if no word count is specified

        # Compute actual word counts
        word_count_actuals = [len(response.split()) for response in decoded_responses]

        # Convert to tensors and ensure gradient tracking
        word_count_targets = torch.tensor(word_count_targets, device=outputs.loss.device, dtype=torch.float)
        word_count_actuals = torch.tensor(word_count_actuals, device=outputs.loss.device, dtype=torch.float)

        # Calculate word count differences
        word_count_differences = Variable(
            torch.abs(word_count_targets - word_count_actuals),
            requires_grad=True
        )

        # Mean word count loss
        word_count_loss = word_count_differences.mean()

        return (word_count_loss, outputs) if return_outputs else word_count_loss

trainer = WordCountLossTrainer(
  model=model,
  peft_config=peft_config,
  tokenizer=tokenizer,
  formatting_func=create_prompt,
  args=args,
  train_dataset=dataset["train"],
  eval_dataset=dataset["validation"]
)

<ipython-input-39-cd5b867e08e8>:10: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `WordCountLossTrainer.__init__`. Use `processing_class` instead.
  super(WordCountLossTrainer, self).__init__(*args, **kwargs)


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Trainer.tokenizer is now deprecated")

trainer.train()

Epoch,Training Loss,Validation Loss
1,710.800000,703.940979
2,712.500000,703.940979
3,703.900000,703.940979
4,680.100000,703.940979
5,707.700000,703.940979


Error occurred for input: ]
50
[END_WORD_COUNT]assistant
Paul Begala says he is appalled by Dick Cheney and his relentless effort to revise history.
Begala: Cheney won't admit any error, misgiving or remorse or apologize for any mistake.
Begala: Cheney misled, lied and did anything to drag America into a war with Iraq.
He says whether Cheney is a liar or fool; thousands of troops are dead; Iraq is a disaster.system

You are a helpful summary chatbot.  Summarize the text provided by the user.
The word count of your response should be less than or equal to the value specified in the WORD_COUNT/END_WORD_COUNT block.user
The Department of Justice announced on Thursday it has reached an agreement with the city of Albuquerque, New Mexico, to address a "pattern or practice of excessive force" by the problem-plagued Albuquerque Police Department. The DOJ said the city has agreed to allow an independent monitor and the courts to oversee reforms at the police department, along with community inp

TrainOutput(global_step=4685, training_loss=704.8175026680897, metrics={'train_runtime': 1678.9857, 'train_samples_per_second': 2.79, 'train_steps_per_second': 2.79, 'total_flos': 2.1723387021754368e+17, 'train_loss': 704.8175026680897, 'epoch': 5.0})

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="Trainer.tokenizer is now deprecated")

trainer.train()

Epoch,Training Loss,Validation Loss
1,710.800000,703.940979
2,712.500000,703.940979


Error occurred for input: ]
50
[END_WORD_COUNT]assistant
Paul Begala says he is appalled by Dick Cheney and his relentless effort to revise history.
Begala: Cheney won't admit any error, misgiving or remorse or apologize for any mistake.
Begala: Cheney misled, lied and did anything to drag America into a war with Iraq.
He says whether Cheney is a liar or fool; thousands of troops are dead; Iraq is a disaster.system

You are a helpful summary chatbot.  Summarize the text provided by the user.
The word count of your response should be less than or equal to the value specified in the WORD_COUNT/END_WORD_COUNT block.user
The Department of Justice announced on Thursday it has reached an agreement with the city of Albuquerque, New Mexico, to address a "pattern or practice of excessive force" by the problem-plagued Albuquerque Police Department. The DOJ said the city has agreed to allow an independent monitor and the courts to oversee reforms at the police department, along with community inp

Epoch,Training Loss,Validation Loss
1,710.800000,703.940979
2,712.500000,703.940979
3,703.900000,703.940979
4,680.100000,703.940979


Error occurred for input: 
[END_WORD_COUNT]assistant
Lake Urmia was once one of the world's largest lakes.
Over the last 20 years it has almost completely dried up.
Iran's water crisis puts security, the economy and public health at risk.
President Rouhani has committed $5 billion to save the lake.system

You are a helpful summary chatbot.  Summarize the text provided by the user.
The word count of your response should be less than or equal to the value specified in the WORD_COUNT/END_WORD_COUNT block.user
(CNN) -- Inmates at a prison in Cuba have made a set of hidden-camera videos to expose the conditions there, publicizing the filth and decrepitude of the facilities despite the risk of retribution. "The conditions here at Combinado del Este are subhuman, and the food is unfit for human consumption," says inmate Douglas Moore, who says he is an American convicted of a drug offense. Because he is an American, he says in the video, "I am singled out for abuse. I cannot count all the tim

In [ ]:
trainer.push_to_hub("Rsr2425/llama3181binstruct_word_count_v1")

In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
merged_model.push_to_hub("Rsr2425/llama3181binstruct_word_count_v1", safe_serialization=True)

In [ ]:
tokenizer.push_to_hub("Rsr2425/llama3181binstruct_word_count_v1")